In [14]:
import dask.dataframe as ddf
import pandas as pd
import json
import os
import numpy as np
import re
import pickle

In [2]:
path = '/home/ubuntu/intermediate-result-new-select-editors/bibliography/bibliography-final-res/'

In [3]:
df = pd.read_parquet(path)

In [5]:
selected_editors = pd.read_parquet('/home/ubuntu/xinrui/selected-editors-based-on-cc').index
kept_users = list(selected_editors)

In [15]:
with open('../../all-data-article-clustering/articles-100-threshold.pkl' ,'rb') as f:
    selected_articles = pickle.load(f) 

In [18]:
df = df[df['page.title'].isin(selected_articles)]

In [19]:
def count_contributions(d):
    res = {}
    for item in d:
        if item not in res:
            res[item] = 1
        else:
            res[item] += 1
    return res

In [20]:
ref_dict = df.groupby(['ref_id']).apply(lambda d: count_contributions(d['contributor']))

In [21]:
ref_dict = pd.DataFrame(ref_dict)

In [22]:
ref_info_top_users_groupped = ref_dict.rename(columns={0:'ref_dict'})

In [23]:
n = len(kept_users)

In [24]:
bibcoupling = np.zeros((n,n))

In [25]:
user_dict = dict(enumerate(x.rstrip() for x in kept_users))
user_dict = dict((y,x) for x,y in user_dict.items())

In [26]:
refs = ref_info_top_users_groupped.index.to_list()

In [28]:
%%time
from itertools import combinations
for ref in refs:
    curr_dict = ref_info_top_users_groupped.loc[ref]['ref_dict']
    if len(curr_dict) < 2:
        continue
    all_user_pairs = list(combinations([*curr_dict],2))
    for pair in all_user_pairs:
        u1 = pair[0]
        u2 = pair[1]
        val = min(curr_dict[u1],curr_dict[u2])
        
        index_1 = user_dict[u1]
        index_2 = user_dict[u2]
        
        bibcoupling[index_1][index_2] += val
        bibcoupling[index_2][index_1] += val
        

CPU times: user 2min 26s, sys: 628 ms, total: 2min 27s
Wall time: 2min 27s


In [29]:
matrix = pd.DataFrame(bibcoupling,index=kept_users,columns=kept_users)

In [30]:
matrix.drop(matrix.loc[matrix.sum(axis=1)==0].index, inplace=True)
matrix.drop(columns=matrix.columns[matrix.sum()==0], inplace=True)

In [31]:
np.fill_diagonal(matrix.values,np.nan)

In [39]:
matrix.to_parquet('result/bibliography.parquet')

In [40]:
matrix.to_csv("result/bibliography-all-data.tsv", sep="\t",encoding='utf-16')